# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1. 사전 확률과 우도 확률의 곱의 합을 증거로 나눠 사후 확률을 계산한다.

### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [20]:
# pip install pandas
import pandas as pd
import re

In [21]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [22]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [23]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

In [24]:
df

,review,sentiment
0,love great exceeded expectations,positive
1,worst purchase have ever made completely useless,negative
2,average nothing special but not terrible either,neutral
3,great who can help but love design highly reco...,positive
4,terrible experience will never buy from poor b...,negative
5,acceptable but expected better not just accept...,neutral
6,absolutely wonderful very satisfied with great,positive
7,quality poor broke after one use terrible enough,negative
8,acceptable for price but there better options ...,neutral
9,great quality fast shipping with wonderful love,positive


기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다.

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다.

In [25]:
# 사전 확률 구하는 코드를 작성해주세요.
sentiment_counts=df['sentiment'].value_counts()

total_reviews=len(df)

prior_proba=sentiment_counts/total_reviews

prior_proba

,count
sentiment,
positive,0.4
negative,0.3
neutral,0.3


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [27]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [28]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [29]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [43]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요
words_count=frequency_matrix.drop('sentiment',axis=1).groupby(df['sentiment']).sum()
total_count=words_count.sum(axis=1)

def conditional_proba(word,sentiment):
  if word not in words_count.columns:
    return 0.0

  word_count=words_count.loc[sentiment, word]
  total=total_count.loc[sentiment]

  return word_count/total

words = ['love', 'great', 'awesome']
sentiments = ['positive', 'negative', 'neutral']

for sentiment in sentiments:
    print(f"\nSentiment: {sentiment}")
    for word in words:
        prob = conditional_proba(word, sentiment)
        if(prob>0):
          print(f"P({word}|{sentiment}) = {prob:.4f}")


Sentiment: positive
P(love|positive) = 0.1154
P(great|positive) = 0.1538

Sentiment: negative

Sentiment: neutral


독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [44]:
# 가능도 구하는 코드를 작성해주세요.
##positive 가능도
prob= conditional_proba('love','positive') * conditional_proba('great','positive')
print(f"P(love, great, awesome| positive) = {prob:.4f}")

##negative 가능도
prob= conditional_proba('love','negative') * conditional_proba('great','negative') * conditional_proba('awesome','negative')
print(f"P(love, great, awesome| negative) = {prob:.4f}")


##neutral 가능도
prob= conditional_proba('love','neutral') * conditional_proba('great','neutral') * conditional_proba('awesome','neutral')
print(f"P(love, great, awesome| neutral) = {prob:.4f}")

P(love, great, awesome| positive) = 0.0178
P(love, great, awesome| negative) = 0.0000
P(love, great, awesome| neutral) = 0.0000


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [42]:
words = ['terrible', 'not', 'never']
sentiments = ['positive', 'negative', 'neutral']

for sentiment in sentiments:
    print(f"\nSentiment: {sentiment}")
    for word in words:
        prob = conditional_proba(word, sentiment)
        if(prob>0):
          print(f"P({word}|{sentiment}) = {prob:.4f}")


Sentiment: positive

Sentiment: negative
P(terrible|negative) = 0.0833
P(never|negative) = 0.0417

Sentiment: neutral
P(terrible|neutral) = 0.0417
P(not|neutral) = 0.0833


In [45]:
import numpy as np
# 최종 확률 구하는 코드를 작성해주세요.
# 첫번째 문장
# P(positive|target_review1)

prob= (conditional_proba('love','positive') * conditional_proba('great','positive'))
print(f"P(love, great, awesome| positive) = {prob:.4f}")

# P(negative|target_review1)

prob= conditional_proba('love','negative') * conditional_proba('great','negative') * conditional_proba('awesome','negative')
print(f"P(love, great, awesome| negative) = {prob:.4f}")

# P(neutral|target_review1)

prob= conditional_proba('love','neutral') * conditional_proba('great','neutral') * conditional_proba('awesome','neutral')
print(f"P(love, great, awesome| neutral) = {prob:.4f}")

# 두번째 문장
# P(positive|target_review2)

prob= conditional_proba('terrible','positive') * conditional_proba('not','positive') * conditional_proba('never','positive')
print(f"P(terrible, not, never| positive) = {prob:.4f}")

# P(negative|target_review2)

prob= conditional_proba('terrible','negative') * conditional_proba('never','negative')
print(f"P(terrible, not, never| negative) = {prob:.4f}")

# P(neutral|target_review2)

prob= conditional_proba('terrible','neutral') * conditional_proba('not','neutral')
print(f"P(terrible, not, never| neutral) = {prob:.4f}")




P(love, great, awesome| positive) = 0.0178
P(love, great, awesome| negative) = 0.0000
P(love, great, awesome| neutral) = 0.0000
P(terrible, not, never| positive) = 0.0000
P(terrible, not, never| negative) = 0.0035
P(terrible, not, never| neutral) = 0.0035


A2-1.   
Target review1의 분류 결과: 긍정적
Target review2의 분류 결과: 중립적, 부정적

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2. review에서 'awesome'과 같이 아예 존재하지 않는 경우의 단어들은 zero probability problem을 만들어 낸다는 것을 알 수 있었습니다. Laplace smoothing은 단어가 긍정적인지 부정적인지 우도를 알아낼 때 특정 조건을 만족하는 단어의 수만 분자로 쓰는 것이 아닌, alpha 값을 추가적으로 더해 zero probability problem을 해결했습니다. 추가적으로 alpha의 목적은 zero probability problem의 해결이기에, 너무 높은 alpha 값은 의미 없는 결과를 초래하며 이는 선호되지 않는 방식입니다. 반대로 가장 선호되는 alpha의 값은 1이라는 것도 조사를 통해 알 수 있었습니다.